## Preprocessing data
This class help to load and preprocess saved cars data

In [1]:
import os, pickle

In [2]:
# make real directory path from list of folders
def make_directory_path(*directories):
    path = ''
    for directory in directories:
        path += directory.rstrip('/') + '/' # cut the "/" char from end of the string, then readd it
    return path

# Load pickle file
def load_pickle(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

In [3]:
class data_preprocessor:
    def __init__(self, main_dir, detail_ext='pkl', image_exts=['jpg','jpeg','png','bmp']):
        self.main_dir = make_directory_path(main_dir)
        self.detail_ext = detail_ext
        self.image_exts = image_exts
        self.load_in()
    
    def load_in(self):
        # read all diretories from the main diretory
        brands = [brand for brand in os.listdir(self.main_dir) if os.path.isdir(self.main_dir + brand)]

        # Brands
        brands_final = []
        for brand in brands:
            brand_dir = make_directory_path(self.main_dir, brand)
            cars = [car for car in os.listdir(brand_dir) if os.path.isdir(brand_dir + car)]

            # Cars
            cars_final = []
            for car in cars:
                car_dir = make_directory_path(brand_dir, car)
                files = [file for file in os.listdir(car_dir) if not os.path.isdir(car_dir + file)]
                
                # Files
                images = []
                details_file = None
                for file in files:
                    file_path = car_dir + file
                    extension = os.path.splitext(file)[1][1:].lower()
                    if extension == self.detail_ext:
                        details_file = file_path
                    elif extension in self.image_exts:
                        images.append(file_path)

                if details_file != None:
                    details = load_pickle(details_file)
                    car_final = {'details': details, 'images': images}
                    cars_final.append(car_final)
                    
            if len(cars_final) > 0:
                brand_final = {'brand': brand.lower(), 'cars': cars_final}
                brands_final.append(brand_final)
        self.brands = brands_final


In [4]:
directory = 'data'
DP = data_preprocessor(directory)
DP.load_in()